In [9]:
import pandas as pd
import numpy as np
import matplotlib as mp
from pathlib import Path
import re

In [3]:
ROOT = Path.cwd()
DATA_DIR = ROOT / "datasets"
IPEDS_DIR = DATA_DIR / "ipeds"
PROCESSED_DIR = IPEDS_DIR / "processed"

In [22]:
files = list(PROCESSED_DIR.glob("*.csv"))

# make a set containing the years of data we have
gradRateYears = {int(re.findall(r"\d{4}", file.name)[0]) for file in files if not re.search("4yr_gradrate_by_state", file.name) == None}
gradRateYears


{2012}

In [23]:
# check to see if there are missing years
startYear = min(gradRateYears)
endYear = max(gradRateYears)

for year in range(startYear, endYear + 1):
    if (not year in gradRateYears):
        raise FileNotFoundError(f"Missing Data for {year}")

In [46]:
# construct a dataframe with columns corresponding to state and rows correspondnig to year

rows = []
for year in range(startYear, endYear + 1):
    oneYearData = pd.read_csv(PROCESSED_DIR / f"class_{year}_4yr_gradrate_by_state.csv")
    row = oneYearData.transpose()
    row.columns = row.iloc[0]
    row = row[1:]

    rows.append(row)

gradRateOverTime = pd.concat(rows)
gradRateOverTime



Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,Georgia,...,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Unknown,US Total,GRAND TOTAL
0.15197564153399662,0.155081,0.149236,0.171163,0.157368,0.209238,0.334058,0.259627,0.243525,0.157146,0.157951,...,0.168374,0.390329,0.282564,0.264944,0.171801,0.22449,0.10508,0.128932,0.206614,0.208058
